In [1]:
# !pip install geopandas
import geopandas as gpd
import pandas as pd


# Calculate travel time isochrone catchments

In [2]:
input_path = "../data/transport/traveltime_api/api_responses_1-2-3hrs.txt"
# save api responses as variables
with open(input_path) as f:
    exec(f.read())

In [3]:
# function to add minute and label properties to api data for later visualizations
def add_travel_time_properties(name, minutes, label):
    isochrone = eval(name)
    for feature in isochrone["features"]:
        feature["properties"]["travel_time_min"] = minutes
        feature["properties"]["travel_time_label"] = label
        
        feature["properties"].pop("search_id", None) # remove 'search_id' since they're all "0" from API

    return {"name": name, "travel_time_min": minutes, "travel_time_label": label, "data": isochrone}

# save list of isocrhone variable names
variable_names = [f"isochrone{i:03d}" for i in range(1, 181)]
# save list of travel times as integers
travel_times = list(range(1, 181))
# save list of travel time labels
travel_labels = [f"{i:03d}_min" for i in travel_times]

# save all data into dictionary
isochrone_dict = {name: add_travel_time_properties(name, minutes, label) for name, minutes, label in zip(variable_names, travel_times, travel_labels)}

In [4]:
# save as individual layers
# set path to export GeoPackage
gpkg_output_path1 = "../data/transport/traveltime_api/Paris_3hr-1min_isochrones.gpkg"
# loop through dictionary and save each isochrone as a layer
for name, entry in isochrone_dict.items():
    # convert the GeoJSON dict to a GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(entry["data"]["features"], crs="EPSG:4326")

    # optional: add global metadata columns for convenience
    gdf["travel_time_min"] = entry["travel_time_min"]
    gdf["travel_time_label"] = entry["travel_time_label"]

    # save to GPKG (each layer named after the isochrone)
    gdf.to_file(gpkg_output_path1, layer=name, driver="GPKG")

In [5]:
# save all into one layer
# Collect all features into one GeoDataFrame
all_gdfs = []
for name, entry in isochrone_dict.items():
    gdf = gpd.GeoDataFrame.from_features(entry["data"]["features"], crs="EPSG:4326")
    gdf["travel_time_min"] = entry["travel_time_min"]
    gdf["travel_time_label"] = entry["travel_time_label"]
    gdf["layer_name"] = name  # optional: keep source layer reference
    all_gdfs.append(gdf)
# Combine into one GeoDataFrame
gdf_all = pd.concat(all_gdfs, ignore_index=True)
# order by travel time
gdf_all = gdf_all.sort_values("travel_time_min").reset_index(drop=True)
gdf_all.set_crs("EPSG:4326", inplace=True)
# save as one file
gpkg_output_path2 = "../data/transport/traveltime_api/Paris_3hr_isochrones_combinedlayers.gpkg"
gdf_all.to_file(gpkg_output_path2, layer="all_isochrones", driver="GPKG")

# Assign travel time to each station in RATP rail network

In [6]:
# load isochrones
isochrones = gpd.read_file("../data/transport/traveltime_api/Paris_3hr_isochrones_combinedlayers.gpkg").to_crs(epsg=2154)
# sort isochrones from largest to smallest (for containment assignment)
isochrones_sorted = isochrones.sort_values(by="travel_time_min", ascending=False)

In [7]:
# load stations
stations = gpd.read_file("../data/transport/RATP/railnetwork_stations_Ile-de-France.gpkg").to_crs(epsg=2154)
# set default travel time for all stations
stations["time_min"] = 181

In [8]:
# assign travel times by containment
for _, row in isochrones_sorted.iterrows():
    poly = row.geometry
    minute_val = int(row["travel_time_min"])
    mask = (stations.within(poly)) & (stations["time_min"] > minute_val)
    stations.loc[mask, "time_min"] = minute_val

In [9]:
# output to geopackage
gpkg_output_path = "../data/transport/RATP/railstations_with_traveltime.gpkg"
stations.to_file(gpkg_output_path, driver="GPKG")